In [5]:
import pandas as pd
from collections import Counter
import pprint as pp
import datetime as dt
from game_engine import wordle_game 

In [6]:
data_folder = "data"
data_file = 'unigram_freq.csv'
results_folder = ""
results_file = "wordle_scores.csv"
game_num = 30

In [11]:
game = wordle_game(game_num, folder=results_folder, filename=results_file)
game.load_data(data_folder, data_file)
game.prep_data()
game.recommend_next(dupe_letters=False)


Words in corpus 333333
Count of 5 Letter Words 39933
Highest frequency 5 letter words
   index   word       count      prob
0     35  about  1226734006  0.017723
1     45  other   978481319  0.014136
2     56  which   810514085  0.011709
3     57  their   782849411  0.011310
4     62  there   701170205  0.010130
5     82  first   578161543  0.008353
6     85  would   572644147  0.008273
7     92  these   541003982  0.007816
8     93  click   536746424  0.007754
9    100  price   501651226  0.007247
Highest frequency letters
[('a', 21942), ('e', 18907), ('o', 14627), ('i', 13749), ('s', 13683), ('r', 12185), ('n', 11447), ('l', 10375), ('t', 9856), ('c', 7422)]

Remaining word count 39933
[['about', 1032.06],
 ['other', 865.24],
 ['which', 0.0],
 ['their', 682.32],
 ['there', 0.0],
 ['first', 440.03],
 ['would', 340.59],
 ['these', 0.0],
 ['click', 0.0],
 ['price', 418.71]]


In [14]:
if game.round_num in [0]:
    allow_dupe_letters = False
else:
    allow_dupe_letters = True

guesses = {
    1: ["their", "ggggc"],
    2: ["major", "gggyc"],
    3: ["flour", "ggccc"],
    4: ["", ""],
    5: ["", ""],
    6: ["", ""],
}

game.new_round(guesses)  # round_num = X)
if guesses[game.round_num][1] != "ccccc" and game.round_num < 6:
    game.recommend_next(allow_dupe_letters, options=20, display=20)



Round Number 3


("Correct= {0: {'is': None, 'is_not': []}, 1: {'is': None, 'is_not': []}, 2: "
 "{'is': 'o', 'is_not': []}, 3: {'is': 'u', 'is_not': ['o']}, 4: {'is': 'r', "
 "'is_not': []}}")
"Good\t= ['r', 'o', 'u']"
"Bad\t= ['t', 'h', 'e', 'i', 'm', 'a', 'j', 'f', 'l']"

Remaining word count 3
[['odour', 0.0], ['scour', 0.0], ['usour', 0.0]]


In [147]:
print(game.results)

{'date': datetime.date(2022, 4, 9), 'winning_round': 0, 'word1': 'arose', 'w1_score': 'yygyg', 'word2': 'cards', 'w2_score': 'gyygy', 'word3': 'start', 'w3_score': 'cccyg', 'word4': '', 'w4_score': '', 'word5': '', 'w5_score': '', 'word6': '', 'w6_score': ''}
